<style>
@import url(https://www.numfys.net/static/css/nbstyle.css);
</style>
<a href="https://www.numfys.net"><img class="logo" /></a>
# Seam carving 

### Examples - Modern
<section class="post-meta">
By Sondre Duna Lundemo, Thorvald M. Ballestad, Jenny Lunde, and Jon Andreas Støvneng
</section>
Last edited: March 26. 2021

---

Вы столкнулись со следующей проблемой: 
Дано изображение со значительным количеством неважного пространства. Вы хотите изменить размер изображения таким образом, чтобы содержимое изображения сохранялось, а удалялись только незначительные фрагменты. 
Существует несколько ситуаций, в которых это может быть актуальным вопросом для рассмотрения. 
Например, если вы создаете веб-страницу, вы, скорее всего, захотите, чтобы содержимое на странице имело примерно тот же размер, что и окно браузера. 
Уменьшение размера окна должно (по крайней мере, до некоторой критической точки) только уменьшить пустое пространство и не искажать содержимое внутри него. 
Другая ситуация, в которой это актуально, - это совместное использование и хранение изображений. 
Если у вас ограниченное пространство для хранения или носитель, на котором вы должны разместить изображение, принимает только изображения определенных размеров, может быть уместно иметь метод изменения размера изображения без искажения пропорций его содержимого. 

Резьба по швам (Seam carving) - алгоритм, который делает именно это; это изменитель размера изображения с учетом содержимого. Хотя сама проблема, по-видимому, не связана ни с физикой, ни с программированием, на самом деле существуют прекрасные аналоги этого алгоритма для физических систем, и он предполагает использование *динамического программирования*. 

(Большая часть реализации основана на том, что представлено в [этой](https://www.youtube.com/watch?v=rpB6zQNsbQU) лекции, прочитанной в курсе вычислительного мышления в Массачусетском технологическом институте)

## Описание алгоритма

Здесь мы опишем фундаментальный алгоритм, который мы должны реализовать для изменения размера изображения. Термины, выделенные курсивом, будут более точно определены в следующих разделах. 

- Выберите *изображение*, которое вы хотите масштабировать в горизонтальном направлении. 
- Выберите количество пикселей, на которое вы хотите масштабировать изображение, $p$.
- Повторите следующие действия $p$ раз:
    * Вычислить *энергию* каждого пикселя на изображении.
    * Найти *шов* сверху вниз изображения с наименьшей энергией.
    * Удалить *шов*.

Интуитивно мы ищем на каждой итерации путь, соединяющий верхнюю и нижнюю части изображения, таким образом, чтобы в пикселях, составляющих путь, содержалось как можно меньше информации.

## Энергетика картинки 

В принципе, основная задача состоит в том, чтобы выяснить, какие пиксели изображения могут быть удалены с наименьшей возможной потерей. Поэтому удобно связать энергетическую функцию, которая измеряет информационное содержание каждого пикселя.

Для начала давайте введем некоторые обозначения и определения, чтобы сделать наши идеи более точными. 
Обозначим через $\mathbf{I}$ изображение размера $n\times m$. Интуитивно мы думаем об изображении как о матрице $n\times m$ скалярных значений, представляющих яркость каждого пикселя изображения. 

\begin{align}
    \mathbf{I} : [1,\dots, n] \times [1,\dots,m] &\to \mathbb{R} \label{eq:img}\\
                                        (i,j)    &\mapsto \mathrm{Brightness}(i,j), \quad(1)
\end{align} 

Мы определяем *яркость пикселя* как среднее значение его красного, зеленого и синего содержимого, причем $1.0$ является самым высоким значением.  

In [ ]:
# импорт полезных пакетов
using  Images, LinearAlgebra, ImageFiltering, Statistics, ImageView, Plots

# загрузка изображения
img_path = "images/wave.png"
image = load(img_path)

In [ ]:
function brightness(img_element::AbstractRGB)
    """
    Функция для получения яркости пикселя
    """
    return mean(img_element.r + img_element.g + img_element.b)
end

Если мы на мгновение подумаем об изображении как о отображении, определенном на прямоугольнике $R \subset \mathbb{R}^2$, а не о конечной области в (1), у нас есть очевидный кандидат на *энергию* изображения. Рассмотрим функцию

$$
E(\mathbf{I}; \mathbf{x}) = \sqrt{\left( \frac{\partial \mathbf{I}}{\partial x} \right)^2 + \left( \frac{\partial \mathbf{I} }{\partial y} \right)^2} \equiv |\boldsymbol{\nabla} \mathbf{I} (\mathbf{x})|,
$$

где $\mathbf{x}\in R \subset \mathbb{R}^2$.
То, что мы ищем, по сути, является мерой того, насколько изменяется яркость пикселя при движении в любом направлении от него, то есть градиентом. 
Дискретным аналогом градиента является так называемый оператор Собеля. 
Это оператор, который оценивает градиент функции в точке. Это делается путем умножения матрицы, соответствующей функции, вычисленной в рассматриваемой точке, и ее ближайших соседей $8$, на следующие матрицы:  

$$
G_x = \frac{1}{8} \begin{pmatrix}
-1 & 0 & 1\\
-2 & 0 & 2\\
-1 & 0 & 1 
\end{pmatrix}
$$
и
$$
G_y = \frac{1}{8}\begin{pmatrix}
-1 & -2 & -1\\
0 & 0 & 0\\
1 & 2 & 1 
\end{pmatrix}.
$$

При умножении матрицы соседей на $G_x$ норма результирующей матрицы будет большой, если справа или слева от центральной точки есть вертикальное ребро. 
В этом отношении $G_x$ обнаруживает вертикальные ребра. Аналогично, $G_y$ обнаруживает горизонтальные ребра. 
*градиент изображения* в пикселе $(i,j)$ аппроксимируется функцией
\begin{equation}\label{eq:conv}
    g(i,j) \equiv \sum_{k =-1}^{+1} \sum_{l = -1}^{+1} \mathbf{G}(k,l) \mathbf{I}(i-k,j-l) \equiv (\mathbf{G} \star \mathbf{I}) (i,j), \quad(2)
\end{equation}
где мы определяем оператор $\mathbf{G}(k,l)$ как 
$$
    \mathbf{G}(k,l) \mathbf{I}(i-k,j-l) \equiv \sqrt{ |G_x(i,j)\mathbf{I}(i-k,j-l)|^2 + |G_y(i,j)\mathbf{I}(i-k,j-l)|^2  }.
$$
Выражение в уравнении (2) принимает форму *свертки*, оправдывая обозначение $\mathbf{G}\star\mathbf{I}$. 
Мы будем использовать свертку в каждой точке в качестве меры энергии изображения, то есть

\begin{equation}\label{eq:energy}
    E(\mathbf{I};i,j) = (\mathbf{G} \star \mathbf{I}) (i,j).\quad(3)
\end{equation}

Однако с уравнением (3) остается небольшое осложнение. Поскольку изображение имеет конечную протяженность, функция в уравнении (2) не определена для $i=1,n$ и $j=1,m$. Этот крайний случай часто обрабатывается путем расширения изображения  заполнением нулей на границах.   

На самом деле, понимание того, что это градиентное приближение является сверткой, полезно для нас, поскольку существуют методы применения таких сверток к изображениям, встроенным в julia (а также в python, если на то пошло). 
Соответственно, мы оставляем это встроенному фильтру для обработки расчета и граничного случая, упомянутого выше. По умолчанию используется нулевое заполнение, упомянутое ранее. Для обсуждения более продвинутых стратегий заполнения мы рекомендуем вам обратиться к [ссылке](https://juliaimages.org/latest/function_reference/).

В julia эти свертки применяются к изображению с помощью функции `imfilter`, аргументами которой являются изображение и ядро, с помощью которого вы хотите свернуть изображение. Ядра Sobel извлекаются из `Kernel.sobel()`. Мы отражаем ядра, так как `imfilter` в julia по умолчанию - это не свертка, а скорее операция корреляции, которая очень похожа. Для получения более подробной информации ознакомьтесь с [материалом](https://juliaimages.org/latest/function_reference/).

In [ ]:
G_y = reflect(Kernel.sobel()[1])
G_x = reflect(Kernel.sobel()[2])

function E(img::Array{RGB{N0f8},2})
    """
    Функция для вычисления энергии изображения
    
    Parameters
    ----------
    img : Array{RGB{N0f8},2}
        Изображение
    
    Returns
    -------
    energy : Float64
        Энергия картинки
    """
    gray_img = brightness.(img)
    ∇x = imfilter(gray_img, G_x)
    ∇y = imfilter(gray_img, G_y)
    
    return sqrt.(∇x.^2 + ∇y.^2)
end 

В ячейке ниже мы тестируем нашу функцию, чтобы увидеть энергетический ландшафт тестового изображения, которое мы импортировали ниже. 
Как и ожидалось, регионы на изображении, которые мы считаем обладающими высоким информационным содержанием, также обладают высокой энергией в этом отношении. 

In [ ]:
# простая функция, используемая для построения изображения
function plot_image(image_array,title)
    gr()
    plt = heatmap(image_array, title=title,yflip = true)
    return plt
end

In [ ]:
e = E(image)
plot(plot_image(e,"Energy landscape"))

Далее нам нужно определить *шов* изображения. В этом блокноте мы будем рассматривать только сжатие изображений в горизонтальном направлении, поэтому нам понадобятся только вертикальные швы. Неофициально вертикальный шов - это некоторый путь от верхней части изображения к нижней. Более формально *вертикальный шов* $n\times m$ - мерного изображения является набором

$$
    s(\mathbf{I}) = \lbrace (x_s(i),i) \rbrace_{i=1}^{n} \subset \mathbb{R}^2
$$

точек на плоскости $\mathbb{R}^2$, чьи последовательные координаты $x$ не могут отличаться более чем на $1$. Следовательно, функция $x : [1,\dots,n] \to [1,\dots,m]$ - это любое отображение, связывающее с любым из вертикальных индексов $n$ один из горизонтальных индексов $m$, при условии ограничения, что $|x(i-1) - x(i)| \leq 1$ for all $i = 2,\dots,n$.

*Внимание*: При переводе индексирующей нотации, используемой в программировании, в математическую нотацию можно заблудиться и запутаться. $0\leq i\leq n$ - это индекс, относящийся к направлению $y$ изображения, в то время как $0\leq j \leq m$ - это индекс, относящийся к направлению $x$. Точки в $\mathbb{R}^2$ записываются в форме $(x(j),y(i))$, в то время как функция $\mathbf{I}$ принимает аргументы в форме $(i, j)$, эффективно действуя так же, как матрица.

Энергия шва - это просто энергия всех пикселей, составляющих шов:
$$
    E(s(\mathbf{I})) =\sum_{i = 1}^{n} E \left(\mathbf{I}; i, x_s(i)\right).
$$

Чтобы найти оптимальный шов, мы ищем шов $s^*$, минимизирующий энергию:
$$
    s^* = \underset{s}{\mathrm{argmin}} \left\lbrace E(s(\mathbf{I}))\right\rbrace  = \underset{\mathbf{s}}{\mathrm{argmin}} \left\lbrace \sum_{i = 1}^{n} E\left(\mathbf{I}; i, x_s(i)\right) \right\rbrace.
$$

Это выполняется с помощью *динамического программирования*. Процедура заключается в следующем [[2]](#article) : 
- Пройдитесь по изображению от второй последней строки (т.е. $i = n - 1$) до вершины и вычислите *кумулятивную* минимальную энергию $M$ для всех возможных швов для каждого пикселя $(i,j)$:

$$
            M(i,j) = E(i,j) + \min\left\lbrace M(i+1,j-1), M(i+1,j), M(i+1,j+1) \right\rbrace .
$$ Функция $M(i,j)$ определяется рекурсивно, с базовым случаем $M(n,j) = E(n,j)$.
- После вычисления $M(i,j)$ для всех швов пиксель $(i, j)$, дающий минимальное значение первой строки $M$, будет началом оптимального шва. 
- Наконец, мы возвращаемся от этого пикселя, чтобы сделать вывод об остальной части оптимального шва.

Мы будем ссылаться на $M$ как на энергетическую карту изображения. Алгоритм реализован в приведенной ниже функции. Обратите внимание, что в оригинальной статье [[2]](#article) они перебирают изображение вниз, в отличие от нас. 

In [ ]:
function energy_map(energy::Array{Float64,2})
    """
    Функция поиска энергетической карты изображения. То есть, обеспечивающая
    информацию о том, в какую сторону идти в каждой точке изображения.
    
    Parameters
    ----------
    
    energy : Array{Float64,2}
        Энергия изображения 
    
    Returns
    -------

    energy_map : Array{Float64,2}
        Энергетическая карта
    
    next_elements : Array{Int64,2}
        Карта направлений, которые нужно выбрать в каждой точке изображения.
    """
    energy_map    = copy(energy)               # карта содержит энергию в каждой точке  
    next_elements = zeros(Int64, size(energy)) 
    
    n,m = size(energy)
    
    for i ∈ n-1:-1:1, j ∈ 1:m 
        # использование min и max, чтобы избежать выхода из изображения
        j_minus = max(j-1,1) # шаг влево
        j_plus  = min(j+1,m) # шаг вправо
        
        pixel_energy, next_element = findmin(energy_map[i+1, j_minus:j_plus]) 
        
        energy_map[i,j] += pixel_energy
        
        # next_element = 1,2,3, поэтому сопоставьте его с -1,0,1, вычитая 2. 
        # Если x_minus == 1, мы можем перейти только к 0,1, поэтому мы добавляем 1
    
        next_elements[i,j] = next_element - 2 + (j_minus==1) 
    end
    return energy_map, next_elements
end 

Прежде чем взглянуть на то, как выглядит энергетическая карта выбранной нами картинки, давайте рассмотрим некоторые построенные примеры, чтобы получить некоторое представление о том, что такое энергетическая карта. Рассмотрим энергетический ландшафт, состоящий из диска с высокой энергией и его окружений с низкой энергией. Очевидно, что мы должны стараться избегать диска, когда выбираем свой путь, и поэтому мы ожидаем, что энергетическая карта уведет пути от центра. Это именно то, что мы наблюдаем ниже.

In [ ]:
# Создание энергетического ландшафта в форме диска
D = zeros((500,500))

for i = 1:500
    for j = 1:500
        if (i-250)^2 + (j-250)^2 <= 100^2
            D[i,j] = 1
        end
    end
end
map_sphere, _ = energy_map(D)
plt1 = plot_image(D,"Disc energy landscape")
plt2 = plot_image(map_sphere,"Energy map of disc landscape")
plot(plt1,plt2,layout = (1,2),size = (900,400))

В качестве другого примера рассмотрим энергетический ландшафт, состоящий из коробки с высокой энергией и низкоэнергетическим окружением. 

In [ ]:
# Создание энергетического ландшафта в форме коробки

box = zeros((500,500))
box[100:400,100:400] .= 1

map_box, _ = energy_map(box)
plt1 = plot_image(box,"Box energy landscape")
plt2 = plot_image(map_box,"Energy map of box landscape")
plot(plt1,plt2,layout = (1,2),size = (900,400))

Такие же "древовидные" структуры наблюдаются и на энергетической карте нашей исходной картинки. 
Эти структуры возникают по существу по той же причине, что и в приведенных выше простых примерах. 
Разница лишь в том, что у нас гораздо больше областей с высокой энергией, поэтому эти структуры накладываются друг на друга, образуя нечто, напоминающее лес. 
Оптимальный путь избегает всех деревьев.  

In [ ]:
map, _ = energy_map(E(image))
plot_image(map,"Energy-path landscape")

## Лирическое отступление

Предположим, что энергетический ландшафт, который мы рассматриваем, является энергетическим ландшафтом химической реакции. Кроме того, предположим, что координата $y$ представляет степень завершения реакции, а координата $x$ представляет некоторый физический параметр реакции. Теперь энергия реакции будет зависеть от пути сверху вниз в нашем ландшафте. То есть при переходе от $0$ к $100\%$ энергия будет зависеть от начального значения параметра $x$ на каждом временном шаге реакции. То, что алгоритм вырезания швов делает в этом случае, заключается в том, чтобы найти наиболее энергетически благоприятный способ завершения реакции, для чего приходится постоянно варьировать $x$. Если пойти еще дальше, можно сказать, что природа решает, как должна протекать реакция, вырезая наименее значимый шов энергетического ландшафта. На самом деле можно утверждать, что между этим алгоритмом и большинством физических систем существуют очевидные связи, поскольку все они ограничены той или иной формой *принципа Гамильтона*: 
$$
   \frac{\delta }{\delta \mathbf{q}} \int_{t_1}^{t_2} \mathrm{d}t L (\mathbf{q},\dot{\mathbf{q}},t) = 0.
$$

### *Замечание*:

Энергетический ландшафт в этом случае, конечно, не был бы похож на нашу картинку, и было бы большим упрощением предположить, что он зависит только от одного параметра. В некотором пространстве более высокого измерения, где одна координата представляет процесс протекания реакции, а другие - параметры реакции, можно было бы представить, что ландшафт будет выглядеть очень сложным, но идея, по сути, та же самая. Еще одна вещь, которую следует упомянуть, заключается в том, что "функция затрат" реакции - это не обязательно энергия как таковая, а скорее некоторый соответствующий термодинамический потенциал. 

## Сшивание кусочков вместе

Чтобы завершить алгоритм, нам нужно
- Функция для получения оптимального шва $s^*$ с учетом энергии. 
- Функция для удаления этого конкретного шва из изображения и карты энергии.
- Функция для повторения вырезания заданное количество раз.

Эти функции записаны в ячейке ниже. Можно было бы подумать, что нам нужно будет только удалить шов из энергии и использовать результат в качестве энергии в следующем расчете. Однако, поскольку энергия в точке зависит от яркости соседей, это будет неверно. Мы, по крайней мере, должны были бы рассчитать энергию всех затронутых пикселей. Чтобы быть в безопасности, можно было бы рассчитать энергию вырезанного изображения на каждом шаге, но для этого требуется выполнить множество вычислений, которые мы уже выполняли на предыдущих шагах. Мы решили решить эту проблему, пересчитав энергию прямоугольной полосы изображения, которая полностью содержит шов и прилегающие к нему швы. 

### _Замечание_:

Обратите внимание, что ниже у нас есть две функции с именем `remove_seam`, параметры которых отличаются. Это похоже на *перегрузку* функций, например, в $\texttt{c++}$, но не совсем то же самое. Обычно перегрузка включает в себя решение на основе типов аргументов и определяет, какая версия функции должна быть вызвана.

In [ ]:
function get_seam(energy::Array{Float64,2})
    """
    Функция для получения оптимального шва с учетом энергетической карты изображения.
    
    Параметры
    ----------
        energy : Array{Float64,2}
            энергия изображения
    
    Возвращается
    -------
        seam : Array{Tuple{Int64,Int64},1}
            Массив кортежей, соответствующих шву. Обратите внимание, что индексация применяется по отношению к
            математической нотации, так как i <-> y, and j <-> x. 
    """
    map, next_elements = energy_map(energy) # вычисляет энергетическую карту изображения
    min_element = argmin(map[1,:])          # находит элемент первой строки на карте, минимизирующий энергию
    
    n       = size(next_elements)[1]
    x       = zeros(Int64,n)
    x[1]    = min_element
    
    # Первый элемент уже введен в шов
    for i = 2:n
        x[i] = x[i-1] + next_elements[i, x[i-1]]
    end
    seam = tuple.(1:n,x)
    return seam
end

function remove_seam(img::Array{RGB{N0f8},2},seam::Array{Tuple{Int64,Int64},1})
    """
    Функция удаления шва с изображения.
    
    Параметры
    ----------
        img : Array{RGB{N0f8}},2}
            Изображение, с которым мы работаем
        seam : Array{Tuple{Int64,Int64},1}
            Шов-кандидат на удаление.
    
    Возвращается
    -------
        new_img : Array{RGB{Normed{UInt8,8}},2}
            Изображение с удаленным швом.
    """
    new_img = img[:,1:end-1]
    for (i,j) ∈ seam
        new_img[i, 1:j-1] .= img[i,1:j-1]
        new_img[i, j:end] .= img[i,j+1:end]
    end
    return new_img
end

function remove_seam(energy::Array{Float64,2},img::Array{RGB{N0f8},2},seam::Array{Tuple{Int64,Int64},1})
    """
    Функция для удаления шва.
    
    Параметры
    ----------
        energy : Array{Float64,2}
            Energy of image
        img : Array{RGB{N0f8},2}
            Image with carved out seam
        seam : Array{Tuple{Int64,Int64},1}
            Seam to remove.
    
    Возвращается
    -------
        new_energy : Array{Float64,2}
            Energy of carved image
    """
    
    n,m = size(img) # size of _new_ image
    
    new_energy = zeros(size(energy[:,1:end-1]))
    j_min, j_max = extrema(last.(seam))
    
    new_energy[:,1:j_min-1]   = energy[:,1:j_min-1]
    new_energy[:,j_max+1:end] = energy[:,j_max+2:end]
    
    # Пересчет энергии (возможно) затронутых пикселей
    # Set to boundary if they happen to be mapped outside the image!
    j_min = j_min - 1 + (j_min == 1)
    j_max = j_max + 1 - 2*(j_max == m + 1) - (j_max == m)
    
    new_energy[:,j_min:j_max] = E(img[:,j_min:j_max])

    return new_energy
end


function seam_carving(img::Array{RGB{N0f8},2},res::Int64)
    """
    Function for carving out a given number of seams of an image.
    
    Параметры
    ----------
    
    img : Array{RGB{Normed{UInt8,8}},2}
        Image.
    res : Int64
        Number of pixels in the horisontal direction in the carved image.
    
    Возвращается
    -------
    img : Array{RGB{Normed{UInt8,8}},2}
        The carved image.
    """
    @assert(res >= 0 || res <= size(img)[2])
    energy = E(img)
    for i = (1:size(img)[2] - res)
        seam   = get_seam(energy)
        img    = remove_seam(img, seam)
        energy = remove_seam(energy,img,seam)
    end    
    return img
end

In [ ]:
# Check the size of the image
print(size(image))

In [ ]:
new_image = seam_carving(image, 700);

In [ ]:
display(new_image)
display(image)

### Замечания: 
- Как я уверен, вы уже подозревали, у этого алгоритма есть некоторые очевидные ограничения. Если мы попытаемся уменьшить размеры изображения за пределы точки, в которой мы можем гарантировать, что никакая информация не будет потеряна, результат будет ужасным. Это можно легко увидеть, выбрав искусственно короткую ширину для картинки.

In [ ]:
bad_carv = seam_carving(image,400);
display(bad_carv)

- Если бы вы действительно попытались запустить ячейку выше, вы бы заметили, что вычисления здесь требуют довольно много времени. Поскольку джулия обычно довольно быстра, можно заподозрить, что в реализации есть некоторые плохие решения. Это, конечно, всегда может быть так, но рассмотрим этот грубый расчет:
    
   Для каждого пикселя изображения мы вычисляем энергию, умножая две матрицы $3\times 3$ с соседями. Это  
```Julia
2 * 3 * 3 * prod(size(image))
```
    шагов вычислений. Для выбранного нами изображения мы должны выполнить 11 273 130 операций только для нахождения энергии изображения на первой итерации! Значительное количество времени также тратится на функцию для получения энергетической карты, но мы не будем здесь рассматривать сложность этой функции. Принимая это во внимание, не следует так удивляться, что эти расчеты занимают некоторое время, хотя на первый взгляд они кажутся вполне невинными. Попробуйте написать алгоритм на python и посмотрите, сможете ли вы заставить его работать так же быстро, как в julia! 

In [ ]:
print("Количество вычислений при расчете энергии изображения: ", 2 * 3 * 3 * prod(size(image)))

In [ ]:
@time seam_carving(image,700);

# References 

 <a name="mitcourse">[1]</a> _Lecture 3 from MIT-course 18.S191_ https://computationalthinking.mit.edu/Fall20/lecture3/ <br>
 <a name="article">[2]</a> Shai Avidan and Ariel Shamir. 2007. _Seam carving for content-aware image resizing._ In ACM SIGGRAPH 2007 papers (SIGGRAPH '07). Association for Computing Machinery, New York, NY, USA, 10–es. DOI:https://doi.org/10.1145/1275808.1276390 <br>
 
 + https://habr.com/ru/post/183638/